In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

def obtener_articulos(text, paginas=100):
    """
    Extrae los nombres de los artículos, URLs y fechas de publicación de las páginas especificadas
    de la búsqueda en El Tiempo sobre migrantes venezolanos en Colombia.

    Parameters:
    text(str): Texto de busqueda del articulo
    paginas(int): Número de páginas a procesar.
    :return: DataFrame con las columnas 'Título', 'Fecha de Publicación' y 'URL'.
    """
    # Lista para almacenar los datos
    datos = []
    try:
        for page in range(paginas):
            url_api = f'https://www.elespectador.com/pf/api/v3/content/fetch/searcherTag?query=%7B%22author%22%3Anull%2C%22date%22%3Anull%2C%22from%22%3A{page}%2C%22keyword%22%3A%22{text}%22%2C%22section%22%3A%5B%22%2Fcolombia%22%5D%2C%22subtype%22%3A%5B%22Art%C3%ADculos%22%5D%7D&d=937&_website=el-espectador'

            response = requests.get(url_api,timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            html_content = soup.string.strip()

            data = json.loads(html_content)

            for element in data.get('content_elements', []):
                title = element.get('headlines', {}).get('basic', {})
                date = element.get('display_date', 'No Date')
                url = element.get('canonical_url', 'not found')
                url = 'https://www.elespectador.com/' + url
                
                datos.append({
                    'title': title,
                    'date': date,
                    'url': url
                })

    except requests.exceptions.RequestException as e:
        print(f'Error al realizar la solicitud: {e}')
    finally:
        df = pd.DataFrame(datos)
        return df


In [4]:
obtener_articulos('migrante-venezolano', paginas=100)

,title,date,url
0,Aumentó el número de migrantes que cruzaron la...,2024-04-01T02:19:33.129Z,https://www.elespectador.com//colombia/mas-reg...
1,Migración Colombia entregará permisos temporal...,2023-12-15T22:39:47.098Z,https://www.elespectador.com//colombia/mas-reg...
2,Asesinaron a tres migrantes venezolanos en Cas...,2023-11-21T18:50:23.188Z,https://www.elespectador.com//colombia/mas-reg...
3,Asesinaron a un extranjero en un hotel del cen...,2023-10-18T02:28:17.866Z,https://www.elespectador.com//colombia/medelli...
4,Habrá feria de empleo para venezolanos en Mede...,2023-10-02T19:09:34.429Z,https://www.elespectador.com//colombia/medelli...
...,...,...,...
820,Gobierno y ONU iniciaron registro de migrantes...,2018-04-06T22:29:59Z,https://www.elespectador.com//colombia/mas-reg...
821,Cúcuta abre primer Centro de Atención Transito...,2018-02-03T23:12:55Z,https://www.elespectador.com//colombia/mas-reg...
822,Gobierno y ONU iniciaron registro de migrantes...,2018-04-06T22:29:59Z,https://www.elespectador.com//colombia/mas-reg...
823,Cúcuta abre primer Centro de Atención Transito...,2018-02-03T23:12:55Z,https://www.elespectador.com//colombia/mas-reg...
